In [1]:
import selenium.webdriver as webdriver
import time
import sqlite3

In [2]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get("https://www.thehindu.com/archive/web/")

In [3]:
def func1(name):    
    """Get list of text in all element by class_name"""
    all_elements = driver.find_elements_by_link_text(name)
    return all_elements

In [4]:
def get_list_by_class_name(class_name):
    """Get list of text in all element by class_name"""
    element_list = []
    try:
        all_elements = driver.find_elements_by_class_name(class_name)
        element_list = [x.text for x in all_elements if len(x.text) > 0]
    except (NoSuchElementException, WebDriverException) as e:
        print(e)
    return element_list

In [5]:
months1 = ['Jan','Feb','Mar','Apr','May','Jun']
months2 = ['Aug','Sep','Oct','Nov','Dec']

In [6]:
### Months is list of links of all months from year 2016 to year 2021 June
Months = []
for month in months1:
    m1 = func1(month)
    for i in range(6,12):     ### Modify range to change years 
        Months.append(m1[i].get_attribute("href"))
        
for month in months2:
    m2 = func1(month)
    for i in range(7,12):
        Months.append(m2[i].get_attribute("href"))
        
July = func1('Jul')
for i in range(6,11):
    Months.append(July[i].get_attribute("href"))   

In [7]:
len(Months)

66

In [5]:
### To establish connection with database db_file
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [10]:
### created SQLITE TABLE date_link_tb which have links for each day between 2016 to 2021 June
### have column processed which is 0 and become 1 if extract all articles link of that day 
curr.execute("""create table date_link_tb(
            date_id INTEGER PRIMARY KEY,
            link TEXT,
            processed INTEGER)""")

In [6]:
### function to insert links of each day in date_link_tb TABLE

def insertDateLinkIntoTable(date_id , link , processed):
    try:
        sqliteConnection = sqlite3.connect('ArticlesDatabase.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sqlite_insert_with_param = """insert into date_link_tb(date_id , link , processed) values (?,?,?);"""

        data_tuple = (date_id , link , processed)
        cursor.execute(sqlite_insert_with_param, data_tuple)
        sqliteConnection.commit()
        print("Python Variables inserted successfully in date_link_tb")

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert Python variable into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")

In [7]:
### function to get link given Text of that link
def get_link_by_text(text):
    """Find link in the page with given text"""
    element = driver.find_element_by_link_text(text.strip())
    
    return element.get_attribute("href")

In [ ]:
###  insert links of each day
date_id = 0
for m in Months:
    driver.get(m)
    time.sleep(2)
    DAYS = get_list_by_class_name("ui-state-default")
    for j in DAYS:
        date_id = date_id + 1
        insertDateLinkIntoTable(date_id,get_link_by_text(j),0)

In [11]:
### created SQLITE table to store links of articles with its title and label 
### there is a column "used" = 0 by default but become 1 if article text is extracted of that day
curr.execute("""create table articles_link_tb(
            art_id INTEGER PRIMARY KEY,
            title TEXT,
            link TEXT,
            label TEXT,
            used INTEGER)""")

In [18]:
### created SQLITE table to store article text with its label 
curr.execute("""create table articles_tb(
            article_id INTEGER PRIMARY KEY,
            article TEXT,
            label TEXT)""")

In [8]:
### function to insert article links with its title and label in sqlite table articles_link_tb
def insertVaribleIntoLinkTable(art_id , title , link, label):
    try:
        sqliteConnection = sqlite3.connect('ArticlesDatabase.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sqlite_insert_with_param = """insert into articles_link_tb(art_id,title,link,label, used) values (?,?,?,?,?);"""

        data_tuple = (art_id,title,link,label,0)
        cursor.execute(sqlite_insert_with_param, data_tuple)
        sqliteConnection.commit()
        print("Python Variables inserted successfully in articles_link_tb")

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert Python variable into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")

In [11]:
### calculate number of rows in date_link_tb which is exactly number of days
conn = create_connection('ArticlesDatabase.db')
curr = conn.cursor()
curr.execute("""SELECT COUNT(*) FROM date_link_tb """)
num = curr.fetchone()
no_days = num[0]
curr.close()

In [12]:
no_days

1987

In [13]:
### function to update table date_link_tb column - processed to 1 
### if the links of all articles of that day are extracted successfully 
def update_finished_date(date_id):
    try:
        sqliteConnection = sqlite3.connect('ArticlesDatabase.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        query = "UPDATE date_link_tb SET processed = 1 WHERE date_id =" + str(date_id)

        cursor.execute(query)
        sqliteConnection.commit()
        print("Python Variables updated successfully in date_link_tb")
        
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to update Python variable into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")
    

In [14]:
### function to extract links of particular day by its date_id and it return 1 if already processed
def get_link_by_date(d):
    try:
        sqliteConnection = sqlite3.connect('ArticlesDatabase.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        query = "SELECT link FROM date_link_tb WHERE date_id = " + str(d) + " AND processed = 0 "
        cursor.execute(query)
        l = cursor.fetchone()
        
        sqliteConnection.commit()
        print("link selected successfully from date_link_tb")
        
        cursor.close()
    except sqlite3.Error as e:
        print("Failed to select link from date_link_tb",e)
    
    finally:
        if l == None:
            return 1
        else:
            return l[0]

In [15]:
### function to get count the number of article links stored in articles_link_tb
def get_art_id():
    try:
        sqliteConnection = sqlite3.connect('ArticlesDatabase.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        query = """SELECT COUNT(*) FROM articles_link_tb """
        cursor.execute(query)
        l = cursor.fetchone()
        count = l[0]

        sqliteConnection.commit()
        print("successfully count the rows in articles_link_tb")

        cursor.close()
    except sqlite3.Error as e:
        print("Failed to count from articles_link_tb",e)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")
        return count


In [ ]:
### this code extract all articles link of 1987 days 
### this code handle No Internet exception 
### when intenet comes just run this block of code and you are good to go.

art_id = get_art_id()

for d in range(no_days):   
    link = get_link_by_date(d)
    if link == 1:
        continue
    try:
        driver.get(link)
        time.sleep(5)    ### wait for 5 seconds to load the page completely
        k = driver.find_element_by_id('subnav-tpbar-latest');
        classes =  k.find_elements_by_tag_name('a');
        
        ## number of classes present in that day 
        num = len(classes)
        
        # this is the last element we need to extract if it loads properly then we proceed further 
        # if not it raise an exception and it then just jump to next day 
        ch = driver.find_element_by_id('section_' + str(num))   
    except :
        continue
        
    ## we are here it means page load completely now it extract all the articles link with coressponding label
            
    for i in range(1,num+1):
        try:
            articles = driver.find_element_by_id('section_' + str(i))
            t = articles.find_element_by_class_name('archive-list')
            articles_list = t.find_elements_by_tag_name('a')
            li = len(articles_list)
        except:
            break
            
        else:
            for j in range(li):
                art_id = art_id + 1
                title = articles_list[j].text
                link = articles_list[j].get_attribute("href")
                label = classes[i-1].text
                insertVaribleIntoLinkTable(art_id , title , link,label)
            
    
    else:  ## it try block work then all the links are extracted of that day now we have to set processed = 1 of that day
        update_finished_date(d)
       